<a href="https://colab.research.google.com/github/Rompil/Advanced-CV/blob/main/Week_1_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv

# Pan shapening algorithms

The task is to make  a high resolution coloured image using one high resolution grayscale image and one low resolution colured image. Also, the low ser image is roteted on an arbitrary angle.

**Plan of the solution**
* Read images
* Restore desired image resolution and orientation with Affine transformations



In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

## Read files to process

These files are different size and with different number of colour channels

In [ ]:
# read files for the task
lo_res_half_path = "/content/sample_data/RGB_half.JPG"
lo_res_quat_path = "/content/sample_data/RGB_quater.JPG"
hi_res_gs_path = "/content/sample_data/GRAY.JPG"
lo_res_half = cv2.imread(lo_res_half_path)
lo_res_quat = cv2.imread(lo_res_quat_path)
hi_res_gs = cv2.imread(hi_res_gs_path)

## Restore desired image resolution and orientation with Affine transformations

The idea behind this approach is to find a key feature set in both images and apply affine transformation to the coloured image to restore a resolution and an orientation based on these ones.

In [ ]:
def transform_image(original_image, transform_image):
    """
    The finction transforms ~transform image~ to ~original image~ based on key feature points and affine transformations
    """
    # Initiate ORB detector
    orb = cv2.ORB_create()
    # find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(transform_image,None)
    kp2, des2 = orb.detectAndCompute(original_image,None)
    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    # Match descriptors.
    matches = bf.match(des1,des2)
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    rows,cols,ch = original_image.shape
    #Only three points are need for the Affine transformation
    pts1 = np.array([kp1[match.queryIdx].pt for match in matches[:3]]).astype(np.float32)
    pts2 = np.array([kp2[match.trainIdx].pt for match in matches[:3]]).astype(np.float32)

    M = cv2.getAffineTransform(pts1,pts2)
    dst = cv2.warpAffine(transform_image,M,(cols,rows)) 
    return dst

In [ ]:
# As far as the original image won't change let's fix one of the parameter in the function
from functools import partial
scale_up_transformation = partial(transform_image, hi_res_gs)
in_image = scale_up_transformation(lo_res_half)

## Pan spectral restoration

Pan sharpening is used to make high resolution coloured images by combining two or more images where one is a high resolution grayscale image e.g. infrared picture and one or more rgb image but with lower resolution. There are a few algorithms how to merge these images. I'll try one of them.

In [ ]:
class PanSharpening:


    def __call__(self, pan_image, rgb_image, W=0.1, method="Brovey"):
        # Firstly, let's normalize two images

        
        rgb = self._scale_and_rotate(pan_image, rgb_image)
        if pan_image.shape[2] > 1:
            # pan_image = np.average(pan_image, axis=2)
            pan_image = pan_image[:,:,0]
        if method == "Brovey":
            return self._brovey(pan_image, rgb)
        else:
            raise NotImplementedError()

    @staticmethod
    def _scale_and_rotate(original_image, transformed_image):
        return transform_image(original_image, transformed_image)

    def _brovey(self, pan, rgb):
        # cv2_imshow(pan)
        # cv2_imshow(rgb)
        R = rgb[:,:,0]
        G = rgb[:,:,1]
        B = rgb[:,:,2]

        all_sum = np.sum(rgb, axis=2) / 3
        print((all_in == all_sum).all())

        r = np.multiply(R, pan/all_sum)[:, :, np.newaxis]
        g = np.multiply(G, pan/all_sum)[:, :, np.newaxis]
        b = np.multiply(B, pan/all_sum)[:, :, np.newaxis]
        # r = (0.5 * R + 0.5 * pan)[:, :, np.newaxis]
        # g = (0.5 * G + 0.5 * pan)[:, :, np.newaxis]
        # b = (0.5 * B + 0.5 * pan)[:, :, np.newaxis]

        image = np.concatenate([r,g,b], axis=2)
        return image
    pass

pan_sharp = PanSharpening()
dst = pan_sharp(hi_res_gs, lo_res_quat)

cv2_imshow(dst)


In [ ]:
t= (115,200)
print(in_image[t])
hi_res_gs[t]
R = in_image[:,:,0]
G = in_image[:,:,1]
B = in_image[:,:,2]
all_in = np.sum(in_image, axis=2)
all_in[t]